# Sourmash Downstream

- Containement analysis performed with nonhost reads in sourmash with the genbank db
- Taxonomy output from sourmash was used for downstream analysis performed in R using the SourmashConsumr package (https://research.arcadiascience.com/pub/resource-sourmashconsumr/)

In [ ]:
# install.packages("remotes")
# remotes::install_github("Arcadia-Science/sourmashconsumr")

library(sourmashconsumr)
library(dplyr)
library(ggplot2)
library(RColorBrewer)
library(gpuR)

setwd("./8_Sourmash/")
path <- "./8_Sourmash/"
list.files()

metadata <- read.csv("Metadata.csv")

# read in the compare file
cpath <- "./compare/Noc_comp.csv"
rumen_compare_df <- read_compare_csv(cpath, sample_to_rownames = F)

# Signatures
# Listing all files in the directory
fpaths <- list.files(path = "./signatures/", full.names = TRUE, pattern = "*.sig")
fpaths <- sort(fpaths)

# Read in the signature files
rumen_signatures_df <- read_signature(fpaths)

# Gathers
# List all gather files in the directory
gpaths <- list.files(path = "./gather/gather_1000_k21/", full.names = TRUE, pattern = "*.csv")
gpaths <- sort(gpaths)

# Read in the gather files
rumen_gather_df <- read_gather(gpaths, intersect_bp_threshold = 3000)

# Taxonomy
# List all tax_annotated files in the directory
tpaths <- list.files(path = "./tax_annotated/tax_1000_k21/", full.names = TRUE, pattern = "*.csv")
tpaths <- sort(tpaths)

# Read in the taxonomy files
rumen_taxonomy_annotate_df <- read_taxonomy_annotate(tpaths, intersect_bp_threshold = 3000, separate_lineage = T)

In [ ]:
# Upset kmer plot to compare signatures
rumen_signatures_df <- rumen_signatures_df %>%
  dplyr::filter(ksize == 21)

rumen_signatures_upset_df <- from_signatures_to_upset_df(signatures_df = rumen_signatures_df)
head(rumen_signatures_upset_df)

signatures_upset <- plot_signatures_upset(upset_df = rumen_signatures_upset_df,
                                          # arguments to ComplexUpset::upset():
                                          stripes = c("cyan"),
                                          name = "RFI",
                                          #max_size = 30,
                                          min_size = 10000000,
                                          themes = ComplexUpset::upset_default_themes(text=element_text(size = 16, color = "orange"),
                                                                                      axis.text.y = element_text(size = 13, color = "blue"),
                                                                                      panel.grid = element_blank()))
signatures_upset

In [ ]:
# Working on Rarefaction Plots
# Rarefaction curve
# filter to a single k-mer size
rumen_signatures_df <- rumen_signatures_df %>%
  dplyr::filter(ksize == 21)

rarefaction_df <- from_signatures_to_rarefaction_df(signatures_df = rumen_signatures_df)
head(rarefaction_df)

rarefaction_plt <- plot_signatures_rarefaction(rarefaction_df = rarefaction_df, fraction_of_points_to_plot = 1)
rarefaction_plt

# adds color to sample name
rarefaction_plt +
  ggplot2::theme_minimal() +
  ggplot2::geom_point(ggplot2::aes(color = name))

# join the metadata data frame with the rarefaction_df
rarefaction_df <- rarefaction_df %>%
  dplyr::left_join(metadata, by = c("name" = "Sample_ID"))

# create and modify a plot
plot_signatures_rarefaction(rarefaction_df = rarefaction_df, fraction_of_points_to_plot = 1) +
  ggplot2::theme_minimal() +
  ggplot2::geom_point(ggplot2::aes(color = RFI))

In [ ]:
# Working on Dimension Reduction to assess sample similarities
rumen_compare_mds_df <- make_compare_mds(rumen_compare_df)
head(rumen_compare_mds_df)

compare_plt <- plot_compare_mds(rumen_compare_mds_df)
compare_plt +
  ggplot2::theme_minimal()

# Add metadata
rumen_compare_mds_df %>%
  dplyr::left_join(metadata, by = c("sample" = "Sample_ID"))

compare_mds <- plot_compare_mds(rumen_compare_mds_df) +
  ggplot2::geom_point(ggplot2::aes(color = metadata$RFI))
compare_mds
ggsave("compare_mds_RNA.tiff", compare_mds, dpi = 300)

plot_compare_heatmap(rumen_compare_df, cexRow = 0.75, cexCol = 0.75)

In [ ]:
# Working on gather output
gather_classified <- plot_gather_classified(gather_df = rumen_gather_df)
gather_classified
ggsave("./z_bacteria_figures/gather_classified_RNA.tiff", gather_classified, dpi = 300)

# Upset gather plot
rumen_gather_upset_df <- from_gather_to_upset_df(gather_df = rumen_gather_df)
head(rumen_gather_upset_df)

gather_upset <- plot_gather_upset(upset_df = rumen_gather_upset_df, 
                                  color_by_database = T, 
                                  gather_df = rumen_gather_df,
                                  min_size = 50)
gather_upset
ggsave("gather_upset_RNA.tiff", gather_upset, dpi = 300)

In [ ]:
# Working on taxonomy output
glom_df <-  tax_glom_taxonomy_annotate(taxonomy_annotate_df = rumen_taxonomy_annotate_df,
                                       tax_glom_level = "species",
                                       glom_var = "f_unique_to_query")

head(glom_df)

rumen_taxonomy_upset_inputs <- from_taxonomy_annotate_to_upset_inputs(taxonomy_annotate_df = rumen_taxonomy_annotate_df, 
                                                                         tax_glom_level = "species")

plot_taxonomy_annotate_sankey(taxonomy_annotate_df = rumen_taxonomy_annotate_df, 
                              tax_glom_level = "family")
ggsave("sankey_RNA.tiff", dpi = 300)

#### Subset per RFI
merged_df <- merge(rumen_taxonomy_annotate_df, metadata, 
                   by.x = "query_name", by.y = "Sample_ID")
least_df <- subset(merged_df, RFI == "Least")
most_df <- subset(merged_df, RFI == "Most")

# For least RFI
sankey_least <- plot_taxonomy_annotate_sankey(taxonomy_annotate_df = least_df, 
                                                tax_glom_level = "phylum")
sankey_least
# ggsave("sankey_least_RNA.tiff", sankey_least, dpi = 300)

# For most RFI
sankey_most <- plot_taxonomy_annotate_sankey(taxonomy_annotate_df = most_df, 
                                                    tax_glom_level = "phylum")
sankey_most
# ggsave("sankey_most_RNA.tiff", sankey_most, dpi = 300)

In [ ]:
# Building a phyloseq object for multiple downstream analysis
library(phyloseq)
rumen_phyloseq <- from_taxonomy_annotate_to_phyloseq(taxonomy_annotate_df = rumen_taxonomy_annotate_df, 
                                                     metadata_df = metadata %>%
                                                     tibble::column_to_rownames("Sample_ID"))
rumen_phyloseq

In [ ]:
# Phyloseq object for DAA at a genome level
# Extract OTU table
otu_table_original <- otu_table(mergedPhyla)

# Transpose the OTU table
otu_table_transposed <- t(otu_table_original)

# Replace the original OTU table in the phyloseq object with the transposed table
mergedPhyla_t <- merge_phyloseq(
  phyloseq(otu_table(otu_table_transposed, taxa_are_rows = FALSE)),
  tax_table(mergedPhyla),
  sample_data(mergedPhyla)
)

### TAXONOMY (Creating the relative abundances)
library(reshape2)
library(phyloseq)
ps_domain <- tax_glom(mergedPhyla_t, "domain", NArm=FALSE); write.csv(otu_table(ps_domain), "./Taxonomy_levels/ps_domain.csv")
ps_phylum <- tax_glom(mergedPhyla_t, "phylum"); write.csv(otu_table(ps_phylum), "./Taxonomy_levels/ps_phylum.csv")
ps_class <- tax_glom(mergedPhyla_t, "class"); write.csv(otu_table(ps_class), "./Taxonomy_levels/ps_class.csv")
ps_order <- tax_glom(mergedPhyla_t, "order"); write.csv(otu_table(ps_order), "./Taxonomy_levels/ps_order.csv")
ps_family <- tax_glom(mergedPhyla_t, "family"); write.csv(otu_table(ps_family), "./Taxonomy_levels/ps_family.csv")
ps_genus <- tax_glom(mergedPhyla_t, "genus", NArm=FALSE); write.csv(otu_table(ps_genus), "./Taxonomy_levels/ps_genus.csv")
ps_species <- tax_glom(mergedPhyla_t, "species", NArm=FALSE); write.csv(otu_table(ps_species), "./Taxonomy_levels/ps_species.csv")
ps_strain <- tax_glom(mergedPhyla_t, "strain"); write.csv(otu_table(ps_strain), "./Taxonomy_levels/ps_strain.csv")

ps_domain.r <- transform_sample_counts(ps_domain, function(x) x / sum(x)); write.csv(otu_table(ps_domain.r), "./Taxonomy_levels/ps_domain.r.csv"); write.csv(tax_table(ps_domain.r), "./Taxonomy_levels/ps_domain.r_tax.csv")
ps_phylum.r <- transform_sample_counts(ps_phylum, function(x) x / sum(x)); write.csv(otu_table(ps_phylum.r), "./Taxonomy_levels/ps_phylum.r.csv"); write.csv(tax_table(ps_phylum.r), "./Taxonomy_levels/ps_phylum.r_tax.csv")
ps_class.r <- transform_sample_counts(ps_class, function(x) x / sum(x)); write.csv(otu_table(ps_class.r), "./Taxonomy_levels/ps_class.r.csv"); write.csv(tax_table(ps_class.r), "./Taxonomy_levels/ps_class.r_tax.csv")
ps_order.r <- transform_sample_counts(ps_order, function(x) x / sum(x)); write.csv(otu_table(ps_order.r), "./Taxonomy_levels/ps_order.r.csv"); write.csv(tax_table(ps_order.r), "./Taxonomy_levels/ps_order.r_tax.csv")
ps_family.r <- transform_sample_counts(ps_family, function(x) x / sum(x)); write.csv(otu_table(ps_family.r), "./Taxonomy_levels/ps_family.r.csv"); write.csv(tax_table(ps_family.r), "./Taxonomy_levels/ps_family.r_tax.csv")
ps_genus.r <- transform_sample_counts(ps_genus, function(x) x / sum(x)); write.csv(otu_table(ps_genus.r), "./Taxonomy_levels/ps_genus.r.csv"); write.csv(tax_table(ps_genus.r), "./Taxonomy_levels/ps_genus.r_tax.csv")
ps_species.r <- transform_sample_counts(ps_species, function(x) x / sum(x)); write.csv(otu_table(ps_species.r), "./Taxonomy_levels/ps_species.r.csv"); write.csv(tax_table(ps_species.r), "./Taxonomy_levels/ps_species.r_tax.csv")
ps_strain.r <- transform_sample_counts(ps_strain, function(x) x / sum(x)); write.csv(otu_table(ps_strain.r), "./Taxonomy_levels/ps_strain.r.csv"); write.csv(tax_table(ps_strain.r), "./Taxonomy_levels/ps_strain.r_tax.csv")

# CLR transformation on phyloseq object
mergedPhyla_t_clr<-microbiome::transform(mergedPhyla_t, "clr")
domain_clr<-microbiome::transform(ps_domain, "clr")
phylum_clr<-microbiome::transform(ps_phylum, "clr")
class_clr<-microbiome::transform(ps_class, "clr")
order_clr<-microbiome::transform(ps_order, "clr")
family_clr<-microbiome::transform(ps_family, "clr")
genus_clr<-microbiome::transform(ps_genus, "clr")
species_clr<-microbiome::transform(ps_species, "clr")
strain_clr<-microbiome::transform(ps_strain, "clr")

write.csv(otu_table(mergedPhyla_t_clr), "./Taxonomy_levels/ps_domain_clr.csv")
write.csv(otu_table(domain_clr), "./Taxonomy_levels/ps_domain_clr.csv")
write.csv(otu_table(phylum_clr), "./Taxonomy_levels/ps_phylum_clr.csv")
write.csv(otu_table(class_clr), "./Taxonomy_levels/ps_class_clr.csv")
write.csv(otu_table(order_clr), "./Taxonomy_levels/ps_order_clr.csv")
write.csv(otu_table(family_clr), "./Taxonomy_levels/ps_family_clr.csv")
write.csv(otu_table(genus_clr), "./Taxonomy_levels/ps_genus_clr.csv")
write.csv(otu_table(species_clr), "./Taxonomy_levels/ps_species_clr.csv")
write.csv(otu_table(strain_clr), "./Taxonomy_levels/ps_strain_clr.csv")

In [ ]:
# At this point, tax_table(ps_species.r) was used to build the 
# phylogenomic tree present in the study for archaea, bacteria, and eukaryotes.
# The viral taxonomy tree was build with assembled viral contigs (vOTUs) that contained RdRP